# read file xlsx : select name img_err save file img_error.txt, dictionary save tọa độ of ảnh bị lôic

In [2]:
lambda x: x*3
a = lambda x: x+3
print(a(1))

4


In [61]:
import pandas as pd

df = pd.DataFrame({'name':['truc', 'thanh','trung'], 'age':[12,13,14], 'text':[0,1,1],'xmax':[23,45,12]}, columns=['name', 'age','text','xmax'])
print(df)

df.to_csv('test.csv')

    name  age  text  xmax
0   truc   12     0    23
1  thanh   13     1    45
2  trung   14     1    12


In [7]:
# df2 = pd.read_csv('test.csv', s)
# print(df2)

In [6]:
for i in range(df.shape[0]):
    row = df.iloc[i]
    print('row:',row)

row: name    truc
age       12
text       0
Name: 0, dtype: object
row: name    thanh
age        13
text        1
Name: 1, dtype: object


In [44]:
name = df['name']
print(name.to_string())
# print(name)
name_text1 = df[df['text']==1][['name']]
print('name_text1:', name_text1.to_string())
print('count name :',len(name_text1))

0     truc
1    thanh
2    trung
name_text1:     name
1  thanh
2  trung
count name : 2


In [46]:
string = ''
for i in range(df.shape[0]):
    a = df.iloc[i,2]
#     print(a)
    if a==1:
        string += df.iloc[i,0] +'\n'
print(string)

thanh
trung



In [47]:
with open('img_error.txt','w') as f:
    f.write(string)
    f.close()
    

# lấy tọa độ để cắt ảnh

In [69]:
dict_coordinate ={}
name =''

for i in range(df.shape[0]):
    a = df.iloc[i,2]
    if a==1:
        name = df.iloc[i,0]
        xmin = df.iloc[i,1]
        xmax = df.iloc[i,3]
        dict_coordinate[name] = [xmin,xmax]
print(dict_coordinate)
print(dict_coordinate.keys())
for name in dict_coordinate.keys():
    if name == 'thanh':
        print('OK')
    print(name)
print(dict_coordinate['thanh'])
print(dict_coordinate['thanh'][0])

{'thanh': [13, 45], 'trung': [14, 12]}
dict_keys(['thanh', 'trung'])
OK
thanh
trung
[13, 45]
13


# Cắt ảnh không bị lỗi , label =0

In [56]:
import os
string = ''
for file_img in os.listdir('./data_text'):
    string += file_img + '\n'

with open('img_normal.txt','w') as f:
    f.write(string)
    f.close()

# Cắt ảnh

In [70]:
import numpy as np
import cv2

path =''
def crop (img,xmin,xmax,y_min,y_max):
    image = cv2.imread(img)
    img_cut = image[xmin:xmax, ymin:ymax]
    cv2.imwrite(os.path.join('',img_cut)

def crop_img():
    for file in os.listdir('./data_text'):
        img = os.path.join(path, file)
#         image = cv2.imread(img)
        for name in dict_coordinate.keys():
            if name == file:
                xmin = dict_coordinate[name][0]
                xmax = dict_coordinate[name][1]
                crop(img, xmin, xmax, y_min, y_max)
                
            
# def crop(img):    
#     image = cv2.imread(img)
#     address = image[165:195,38:439]
#     cv2.imwrite('image_cut/cut.png',address)

# Làm mượt ảnh

In [ ]:

import cv2

def processing(img):
    image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    print(image.shape)
    blur = cv2.bilateralFilter(image, 5, 75, 75)
    img_binary = cv2.adaptiveThreshold(blur, 
                                       maxValue=255, 
                                       adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C, 
                                       thresholdType=cv2.THRESH_BINARY,
                                       blockSize=11,
                                       C=6)
    blur_next = cv2.GaussianBlur(img_binary,(3,3),0)
    imgScale = cv2.resize(blur_next, (int(1280), int(64)), interpolation = cv2.INTER_LINEAR)
    print(imgScale.shape)
#     cv2.imwrite('image_address/address.png', imgScale)

# MODEL SEAMESE NETWORK

In [71]:
def get_siamese_model(input_shape):
    """
        Model architecture
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net